In [122]:
import mlflow
from mlflow.client import MlflowClient
import pandas as pd
import os
from overfit.env_settings import settings
import numpy as np


In [9]:
client = MlflowClient(tracking_uri=settings.MLFLOW_TRACKING_URI)


In [167]:
experiment_names = [
    "IMAGENET1K-4-50",
    "IMAGENET1K-5-50",
    "IMAGENET1K-6-75",
    "D4-50Mresnet34C0.1WD0.2LR0.4M0.1",
    "D5-50Mresnet34C0.1WD0.2LR0.4M0.1",
    "D6-75Mresnet34C0.1WD0.2LR0.4M0.1"
]
experiment = client.get_experiment_by_name(experiment_names[4])
assert experiment is not None
experiment.name


'D5-50Mresnet34C0.1WD0.2LR0.4M0.1'

In [168]:
assert experiment is not None
runs = client.search_runs([experiment.experiment_id])


In [169]:
src_errors = []
tgt_errors = []
pimps = []
for run in runs:
    N = int(run.data.params["Frames"])
    filename = run.data.params["Filename"]
    src_acc_top1 = run.data.metrics["Source Accumulated Top-1 Count"]
    tgt_acc_top1 = run.data.metrics["Target Accumulated Top-1 Count"]
    pimp = (
        100 * (tgt_acc_top1 - src_acc_top1) / (src_acc_top1) if src_acc_top1 != 0 else 0
    )
    src_error = 100 * (N - src_acc_top1) / N
    tgt_error = 100 * (N - tgt_acc_top1) / N
    assert src_error >= 0, (src_error, src_acc_top1, N, filename)
    assert tgt_error >= 0, (tgt_error, tgt_acc_top1, N, filename)
    pimps.append(pimp)
    src_errors.append(src_error)
    tgt_errors.append(tgt_error)

df = pd.DataFrame(
    {
        "Source Errors": src_errors,
        "Target Errors": tgt_errors,
        # "Percentage Change": pimps,
    }
)
df.describe()


,Source Errors,Target Errors
count,976.000000,976.000000
mean,81.950797,77.586133
std,17.297296,22.294831
min,0.000000,0.000000
25%,73.846154,66.153846
50%,86.153846,84.615385
75%,95.384615,95.384615
max,100.000000,100.000000


In [49]:
for run in runs[:100]:
    N = int(run.data.params["Frames"])
    src_acc_top1 = run.data.metrics["Source Accumulated Correct Count"]
    tgt_acc_top1 = run.data.metrics["Target Accumulated Correct Count"]
    if src_acc_top1 > tgt_acc_top1:
        print(run.data.params["Filename"])
        print(src_acc_top1, tgt_acc_top1)


data/video/994-stinkhorn-5-65.mp4
30.0 29.0
data/video/984-rapeseed-5-65.mp4
33.0 32.0
data/video/979-valley-5-65.mp4
16.0 1.0
data/video/976-promontory-5-65.mp4
8.0 6.0
data/video/970-alp-5-65.mp4
29.0 16.0
data/video/968-cup-5-65.mp4
4.0 2.0
data/video/964-potpie-5-65.mp4
41.0 40.0
data/video/962-meat_loaf-5-65.mp4
14.0 13.0
data/video/961-dough-5-65.mp4
7.0 4.0
data/video/941-acorn_squash-5-65.mp4
9.0 7.0
data/video/935-mashed_potato-5-65.mp4
16.0 13.0
data/video/917-comic_book-5-65.mp4
16.0 15.0
data/video/910-wooden_spoon-5-65.mp4
10.0 8.0
